In [28]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from transformers import pipeline
from sklearn.model_selection import train_test_split
import tqdm

In [ ]:
import torch
import gc
# Clear cache

# For debugging purposes, check memory stats
print(f"Memory Allocated: {torch.cuda.memory_allocated() / 1e6} MB")
print(f"Memory Reserved: {torch.cuda.memory_reserved() / 1e6} MB")
torch.cuda.empty_cache()
gc.collect()
torch.cuda.reset_peak_memory_stats()
model_path = "andrijdavid/Llama-3-1B-Base"
llama3_model_path = "meta-llama/Llama-3.2-1B"
file_path = "./data/combined_dataset.csv"

def create_input_text(row):
    return (
        f"""You are a member of the IELTS essay evaluation committee.
        Your task is to evaluate the essay based on the given prompt and assign it a score
        between 4 and 9 (in 0.5 increments). There is also a '<4' class, thus a total of 12 classes.
        Provide only the score or '<4' as your response. Think step by step why this essay is good or bad.
        "Prompt: {row['prompt']}\nEssay: {row['essay']}"""
    )

def map_band_to_class(band):
    if band ==  "<4":
        return band_to_class['<4']
    return band_to_class[str(band)]

def tokenize_function(example):
    return tokenizer(
        example["input_text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

band_classes = ['<4'] + ['4', '4.5', '5', '5.5', '6', '6.5', '7', '7.5', '8', '8.5', '9'] 
band_to_class = {band: i for i, band in enumerate(band_classes)}  # Band → Class
class_to_band = {i: band for band, i in band_to_class.items()}  # Class → Band

# Map bands to classes
def map_band_to_class(band):
    if band ==  "<4":
        return band_to_class['<4']
    return band_to_class[str(band)]

def tokenize_function(example):
    return tokenizer(
        example["input_text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

Memory Allocated: 0.0 MB
Memory Reserved: 0.0 MB


In [21]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # Path to your model directory
# model_path = "andrijdavid/Llama-3-1B-Base"

# # Load the tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(llama3_model_path)
# model = AutoModelForCausalLM.from_pretrained(llama3_model_path)

# # Test the model
# prompt = "The key to life is"
# inputs = tokenizer(prompt, return_tensors="pt")
# outputs = model.generate(**inputs, max_length=50)

# # Decode the generated text
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Prepare Test Dataset

In [22]:
# import pandas as pd
# from datasets import Dataset
# from sklearn.model_selection import train_test_split

# # Load the dataset
# file_path = "./data/combined_dataset.csv"
# data = pd.read_csv(file_path).head(10)

# tokenizer = AutoTokenizer.from_pretrained(llama3_model_path)
# tokenizer.pad_token = tokenizer.eos_token

# # Combine 'prompt' and 'essay' into a single input text

# # Apply this to your dataset
# data['input_text'] = data.apply(create_input_text, axis=1)

# data['labels'] = data['band'].apply(map_band_to_class)
# data = data.drop(columns=["evaluation", "band", "prompt", "essay"])

# test_dataset = Dataset.from_pandas(data)


# test_dataset = test_dataset.map(tokenize_function, batched=True)
# test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


In [23]:
# Check the content of test_dataset
# t = test_dataset[0]
#print(test_dataset[0])

# Check the length of test_dataset
# print(f"Length of test_dataset: {len(test_dataset)}")
# for k, v in class_to_band.items():
#     print(f"Class {k} → Band {v}")

#### Prepare Model and Tokenizer, Move them to CUDA.

In [24]:
# num_labels = 12  # Total number of unique band scores
# # C:\Users\26597\.cache\huggingface\hub Models are here
# model_path = "andrijdavid/Llama-3-1B-Base"
# llama3_model_path = "meta-llama/Llama-3.2-1B"

# model = AutoModelForSequenceClassification.from_pretrained(
#     llama3_model_path,
#     num_labels=num_labels,
# )
# t = model.config.eos_token_id
# model.config.pad_token_id = tokenizer.pad_token_id

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
# print(f"Device: {device}")

In [25]:
# print("Tokenizer special tokens:", tokenizer.special_tokens_map)
# print("Model pad_token_id:", model.config.pad_token_id)

In [26]:
# def evaluate_model_on_gpu(model, dataset, batch_size=1):
#     model.eval()  # Set model to evaluation mode
#     predictions = torch.tensor([], dtype=torch.long, device=device)  # Initialize on GPU
#     labels = torch.tensor([], dtype=torch.long, device=device)  # Initialize on GPU

#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
#     for batch in dataloader:
#         with torch.no_grad():
#             # Move batch to GPU
#             input_ids = batch["input_ids"].to(device)
#             attention_mask = batch["attention_mask"].to(device)
#             labels_batch = batch["labels"].to(device)

#             # Get model outputs
#             outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#             logits = outputs.logits
#             predicted_classes = logits.argmax(dim=1)  # Get class with highest logit

#             # Append predictions and labels (on GPU)
#             predictions = torch.cat((predictions, predicted_classes))
#             labels = torch.cat((labels, labels_batch))

#     return predictions, labels

# # Perform evaluation
# predictions, true_labels = evaluate_model_on_gpu(model, test_dataset)

# accuracy = (predictions == true_labels).float().mean().item()
# print(f"Zero-shot accuracy of the pretrained model on the first 100 rows: {accuracy:.2%}")
# print(f"Predictions: {predictions}")
# print(f"True Labels: {true_labels}")

In [ ]:
# fpath = "/Users/barrychen/Desktop/IELTSWritingHelper/datasets_ready/Task_Achievement.csv"
# df = pd.read_csv(fpath)
# df['score'] = df['score'].round(1)
# df_filtered = df[(df['score'] > 1.5) & (df['score'] < 12.0)]

# reverse_mapping_3 = {
#     3.5: 0, 4.0: 0,
#     4.5: 1, 5.0: 1,
#     5.5: 2, 6.0: 2,
#     6.5: 3, 7.0: 3,
#     7.5: 4, 8.0: 4,
#     8.5: 5, 9.0: 5
# }

# df_filtered['score'] = df_filtered['score'].map(reverse_mapping_3)

# df_sampled = df_filtered.groupby('score').sample(
#     n=290, 
#     random_state=42
# ).reset_index(drop=True)

# df_sampled['score'] = df_sampled['score'].astype(int)

# dataset = Dataset.from_pandas(df_sampled)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/barrychen/Desktop/IELTSWritingHelper/datasets_ready/Task_Achievement.csv'

In [ ]:
data = pd.read_csv(file_path).head(200)
data['input_text'] = data.apply(create_input_text, axis=1)
data['labels'] = data['band'].apply(map_band_to_class)
# Drop unnecessary columns. Might need them later
data = data.drop(columns=["evaluation", "band", "prompt", "essay"])

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Prepare datasets for Hugging Face Trainer
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

print(f"Train dataset length: {len(train_dataset)}, Test dataset length: {len(test_dataset)}")

tokenizer = AutoTokenizer.from_pretrained(llama3_model_path)
tokenizer.pad_token = tokenizer.eos_token

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
print(f"train_dataset columns: {train_dataset.column_names}")
print(f"train_dataset[0]: {train_dataset[0]}, train_dataset['labels'][0]: {train_dataset['labels'][0]}")

Train dataset length: 80, Test dataset length: 20


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

train_dataset columns: ['input_text', 'labels', '__index_level_0__', 'input_ids', 'attention_mask']
train_dataset[0]: {'input_text': 'You are a member of the IELTS essay evaluation committee.\n        Your task is to evaluate the essay based on the given prompt and assign it a score\n        between 4 and 9 (in 0.5 increments). There is also a \'<4\' class, thus a total of 12 classes.\n        Provide only the score as your response. Think step by step why this essay is good or bad.\n        "Prompt: Interviews form the basic selecting criteria for most large companies.  However, some people think that interview is not a reliable method of  choosing whom to employ and there are other better methods. To what   extent do you agree or disagree?\nEssay: Hiring employees is the toughest and time-consuming task.  Recruiting a good and hardworking employee can take the company\'s  profits to the next level. Whilst, companies follow a general   tend to take interviews before hiring any employe

In [31]:
print(f"Memory Allocated: {torch.cuda.memory_allocated() / 1e6} MB")
print(f"Memory Reserved: {torch.cuda.memory_reserved() / 1e6} MB")
torch.cuda.empty_cache()
gc.collect()

Memory Allocated: 0.0 MB
Memory Reserved: 0.0 MB


374

## Setting up Trainer

In [32]:
num_labels = 12  # Total number of unique band scores
# C:\Users\26597\.cache\huggingface\hub Models are here
model_path = "andrijdavid/Llama-3-1B-Base"
llama3_model_path = "meta-llama/Llama-3.2-1B"

model = AutoModelForSequenceClassification.from_pretrained(
    llama3_model_path,
    num_labels=num_labels,
)
t = model.config.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Freeze the base model
for param in model.base_model.parameters():
    param.requires_grad = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Device: {device}")
# # Test the model
# prompt = "The key to life is"
# inputs = tokenizer(prompt, return_tensors="pt")
# outputs = model.generate(**inputs, max_length=30)

# Decode the generated text
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Device: cuda


In [33]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    fp16=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    print(f"True Labels: {labels}")
    print(f"Predicted Labels: {preds}")
    print(f"Accuracy: {acc}")
    return {"accuracy": acc}
    
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


# Fine-tune the model
trainer.train()

# Evaluate on the test dataset
test_results = trainer.evaluate()
print(f"Test Results: {test_results}")

  0%|          | 0/15 [00:00<?, ?it/s]

c:\Users\26597\miniconda3\envs\ece1786\lib\site-packages\transformers\models\llama\modeling_llama.py:602: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/5 [00:00<?, ?it/s]

True Labels: [8 3 3 7 8 6 5 9 2 8 4 7 1 6 3 1 7 8 8 5]
Predicted Labels: [ 9  2 10 10 10 10  5  4 10  9  5 10 10  5  7  3  5  3 10 11]
Accuracy: 0.05
{'eval_loss': 2.9854493141174316, 'eval_accuracy': 0.05, 'eval_runtime': 0.9341, 'eval_samples_per_second': 21.41, 'eval_steps_per_second': 5.352, 'epoch': 1.0}


  0%|          | 0/5 [00:00<?, ?it/s]

True Labels: [8 3 3 7 8 6 5 9 2 8 4 7 1 6 3 1 7 8 8 5]
Predicted Labels: [ 9  2  8  0  0  8  5  4  8  8  5 10  9  3  7  3  8  3  8  9]
Accuracy: 0.15
{'eval_loss': 2.539257764816284, 'eval_accuracy': 0.15, 'eval_runtime': 0.935, 'eval_samples_per_second': 21.39, 'eval_steps_per_second': 5.348, 'epoch': 2.0}


  0%|          | 0/5 [00:00<?, ?it/s]

True Labels: [8 3 3 7 8 6 5 9 2 8 4 7 1 6 3 1 7 8 8 5]
Predicted Labels: [9 2 8 0 0 8 5 4 7 8 5 7 7 3 7 3 8 3 8 9]
Accuracy: 0.2
{'eval_loss': 2.4266114234924316, 'eval_accuracy': 0.2, 'eval_runtime': 1.0069, 'eval_samples_per_second': 19.862, 'eval_steps_per_second': 4.966, 'epoch': 3.0}
{'train_runtime': 142.0248, 'train_samples_per_second': 1.69, 'train_steps_per_second': 0.106, 'train_loss': 3.279863484700521, 'epoch': 3.0}


  0%|          | 0/5 [00:00<?, ?it/s]

True Labels: [8 3 3 7 8 6 5 9 2 8 4 7 1 6 3 1 7 8 8 5]
Predicted Labels: [9 2 8 0 0 8 5 4 7 8 5 7 7 3 7 3 8 3 8 9]
Accuracy: 0.2
Test Results: {'eval_loss': 2.4266114234924316, 'eval_accuracy': 0.2, 'eval_runtime': 1.2131, 'eval_samples_per_second': 16.487, 'eval_steps_per_second': 4.122, 'epoch': 3.0}
